<a href="https://colab.research.google.com/github/safikhanSoofiyani/CS6910-Assignment-2/blob/main/CS6910-Assignment2-PartB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import os
import math

# Importing libraries related to Image Preprocessing
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

# Importing libraries related to CNN Model building
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D,Activation,Dropout,BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import ResNet50, InceptionV3, InceptionResNetV2, Xception

random.seed(hash("seriously you compete with me") % 2**32 - 1)
np.random.seed(hash("i am mohammed safi") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)
tf.random.set_seed(hash("ur rahman khan") % 2**32 - 1)

In [ ]:
!pip install -q wandb
import wandb
from wandb.keras import WandbCallback

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 65.8 MB/s 
     |████████████████████████████████| 144 kB 69.6 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
input_image_shape = (256, 256, 3)

In [ ]:
def get_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"

    
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        rescale=1./255,
        validation_split=0.1
    )

    train_dataset=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_dataset=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    #return train_aug, valid_aug

    
    #return train_data, valid_data
    return train_dataset, valid_dataset

In [ ]:
def get_augmented_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path=r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest",
                                        validation_split = 0.1)

    # Validation data is not being augmented
    validation_data_augmentation=ImageDataGenerator(
        rescale=1./255,
        validation_split=0.1
    )

    train_aug=training_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='training')
    valid_aug=validation_data_augmentation.flow_from_directory(path,shuffle=True,seed=19,subset='validation')

    return train_aug, valid_aug

In [ ]:
def build_model(model_name, dense_activation, dense_layer, dropout, trainable_layers):
    '''
    define a keras sequential model based on a pre-trained model intended to be fine tuned
    '''
    input_image_shape = (256, 256, 3)
    input_layer = Input(shape = input_image_shape)

    # add a pretrained model without the top dense layer
    if model_name == 'ResNet50':
      pretrained_model = ResNet50(include_top = False, weights='imagenet',input_tensor = input_layer)
    elif model_name == 'InceptionV3':
      pretrained_model = InceptionV3(include_top = False, weights='imagenet',input_tensor = input_layer)
    elif model_name == 'InceptionResNetV2':
      pretrained_model = InceptionResNetV2(include_top = False, weights='imagenet',input_tensor = input_layer)
    else:
      pretrained_model = Xception(include_top = False, weights='imagenet',input_tensor = input_layer)
    
    #freeze all layers
    for layer in pretrained_model.layers:
        layer.trainable=False 
    
    #set some of the top layers as trainable
    if trainable_layers != 0:
      for layer in pretrained_model.layers[-trainable_layers:]:
        layer.trainable=True

    model = Sequential()
    model.add(pretrained_model)#add pretrained model
    model.add(Flatten()) # The flatten layer is essential to convert the feature map into a column vector
    model.add(Dense(dense_layer, activation=dense_activation))#add a dense layer
    model.add(Dropout(dropout)) # For dropout
    model.add(Dense(10, activation="softmax"))#softmax layer

    return model

In [ ]:
def wandb_train():
   # initializing wandb with the above configuration

    config_defaults = {
        "model_name": "InceptionV3",
        "data_augmentation": True,
        "dense_layer": 128,
        "dropout": 0.0,
        "trainable_layers": 0,
        "batch_size": 64,
        "num_epochs": 10
        
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # collecting the configuration information
    config = wandb.config

    # Local variables, values obtained from wandb config
    model_name = config.model_name
    data_augmentation = config.data_augmentation
    dense_layer = config.dense_layer
    dropout = config.dropout
    trainable_layers = config.trainable_layers
    batch_size = config.batch_size
    num_epochs = config.num_epochs
    dense_activation = "relu"
    
    
    # run name
    run_name='Model_{}_trainable_{}_Data_aug_{}_dropout_{}_dense_layer_{}'.format(config.model_name,
                                                                             config.trainable_layers,
                                                                             config.data_augmentation,
                                                                             config.dropout,
                                                                             config.dense_layer,                                                                       
                                                                             )

    wandb.run.name = run_name

    # Create the data generators

    if data_augmentation == False:
        train_data, validation_data = get_data()
    else:
        train_data, validation_data = get_augmented_data()
    
    # Define the model
    model = build_model(model_name, dense_activation, dense_layer, dropout, trainable_layers)


    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.CategoricalCrossentropy(name='loss'),
                metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')])


    # To save the model with best validation accuracy
    mc = ModelCheckpoint('/content/drive/MyDrive/nature_12K/best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)

    history = model.fit(train_data,
                        validation_data = validation_data,
                        epochs=num_epochs, 
                        callbacks=[WandbCallback()]
                        )
    
    
    # Meaningful name for the run

    wandb_log = True
    wandb.run.finish()
    return history

In [ ]:
hyperparameters = {

        "model_name":{
            'values': ["InceptionV3", "ResNet50", "InceptionResNetV2", "Xception"]
        },

        "data_augmentation": {
            "values": [True, False]
        },

        "dense_layer": {
            "values": [64, 128, 256, 512]
        },

        "dropout": {
            "values": [0.0, 0.1, 0.2, 0.3]
        },
        
        "trainable_layers": {
            "values": [0, 10, 15, 20]
        },

        "batch_size": {
            "values": [64, 128]
        },

        "num_epochs":{
            "values": [5, 10, 15]
        }
        
    }

In [ ]:
!wandb login --relogin
entity_name = "safi-vamsi-cs6910"
project_name = "Assignment 2"

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_config = {

  "method": "bayes",
  "metric": {
      "name":"val_acc",
      "goal": "maximize"
  },
  
  "parameters": hyperparameters
}

sweep_id = wandb.sweep(sweep_config, entity=entity_name, project=project_name)
wandb.agent(sweep_id, wandb_train)

acc,▁█▂▅
epoch,▁▃▆█
loss,█▁▁▁
val_acc,▁▁█▁
val_loss,▁▁█▁
acc,0.09778
best_epoch,3
best_val_loss,2.30258
epoch,3
loss,2.30304
val_acc,0.1001


Create sweep with ID: rrfukwss
Sweep URL: https://wandb.ai/safi-vamsi-cs6910/Assignment%202/sweeps/rrfukwss


wandb: Agent Starting Run: kdat1bys with config:
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.3
wandb: 	model_name: Xception
wandb: 	num_epochs: 15
wandb: 	trainable_layers: 10


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/15
282/282 [==============================] - 261s 914ms/step - loss: 1.1517 - acc: 0.6413 - val_loss: 0.7023 - val_acc: 0.7798 - _timestamp: 1648608276.0000 - _runtime: 265.0000
Epoch 2/15
282/282 [==============================] - 255s 905ms/step - loss: 0.7767 - acc: 0.7487 - val_loss: 0.6667 - val_acc: 0.7748 - _timestamp: 1648608531.0000 - _runtime: 520.0000
Epoch 3/15
282/282 [==============================] - 257s 913ms/step - loss: 0.6492 - acc: 0.7880 - val_loss: 0.6509 - val_acc: 0.7778 - _timestamp: 1648608795.0000 - _runtime: 784.0000
Epoch 4/15
282/282 [==============================] - 258s 915ms/step - loss: 0.5578 - acc: 0.8172 - val_loss: 0.6627 - val_acc: 0.8028 - _timestamp: 1648609057.0000 - _runtime: 1046.0000
Epoch 5/15
282/282 [==============================] - 257s 910ms/step - loss: 0.4878 - acc: 0.8370 - val_loss: 0.6446 - val_acc: 0.8008 - _timestamp: 1648609307.0000 

acc,▁▃▄▅▅▆▆▇▇▇▇████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▅▄▃▃▂▂▂▂▂▁▁▁▁
val_acc,▂▁▁▅▅▄▅▅▂█▅▄▄▅▆
val_loss,▂▁▁▁▁▂▃▃▅▄▆▆███
acc,0.95156
best_epoch,4
best_val_loss,0.64464
epoch,14
loss,0.14552
val_acc,0.80981


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: fbg31x7q with config:
wandb: 	batch_size: 64
wandb: 	data_augmentation: True
wandb: 	dense_layer: 512
wandb: 	dropout: 0.3
wandb: 	model_name: Xception
wandb: 	num_epochs: 15
wandb: 	trainable_layers: 20


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/15
282/282 [==============================] - 258s 906ms/step - loss: 1.1391 - acc: 0.6392 - val_loss: 0.7349 - val_acc: 0.7758 - _timestamp: 1648657081.0000 - _runtime: 262.0000
Epoch 2/15
282/282 [==============================] - 263s 931ms/step - loss: 0.7281 - acc: 0.7607 - val_loss: 0.6721 - val_acc: 0.8098 - _timestamp: 1648657354.0000 - _runtime: 535.0000
Epoch 3/15
282/282 [==============================] - 258s 915ms/step - loss: 0.5978 - acc: 0.8039 - val_loss: 0.6808 - val_acc: 0.8058 - _timestamp: 1648657616.0000 - _runtime: 797.0000
Epoch 4/15
282/282 [==============================] - 252s 894ms/step - loss: 0.5115 - acc: 0.8313 - val_loss: 0.6170 - val_acc: 0.8148 - _timestamp: 1648657862.0000 - _runtime: 1043.0000
Epoch 5/15
282/282 [==============================] - 254s 898ms/step - loss: 0.4153 - acc: 0.8634 - val_loss: 0.6616 - val_acc: 0.8128 - _timestamp: 1648658131.0000 

acc,▁▄▅▅▆▆▇▇▇▇█████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▄▃▃▂▂▂▂▁▁▁▁▁
val_acc,▁▇▆██▆▄█▆▇▄▆█▇▇
val_loss,▃▂▂▁▂▃▅▄▄▄▅▆▄▆█
acc,0.96411
best_epoch,3
best_val_loss,0.61702
epoch,14
loss,0.11564
val_acc,0.81081


wandb: Agent Starting Run: 79cbxt03 with config:
wandb: 	batch_size: 64
wandb: 	data_augmentation: False
wandb: 	dense_layer: 128
wandb: 	dropout: 0.3
wandb: 	model_name: Xception
wandb: 	num_epochs: 15
wandb: 	trainable_layers: 20


Found 9000 images belonging to 10 classes.
Found 999 images belonging to 10 classes.
Epoch 1/15
140/282 [=============>................] - ETA: 1:05 - loss: 1.1525 - acc: 0.6254

In [ ]:
best_hyperparameters = {
      'batch_norm': True,
      'num_filters': 32,
      'filter_org': 2,
      'dropout': 0.4,
      'data_augmentation': False,
      'num_epochs' : 10,
      'batch_size': 128,
      'dense_layer': 512,
      'learning_rate': 0.0001,
      'kernel_size': 3
}

In [ ]:
def get_test_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path_train = r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    path_test = r"/content/drive/MyDrive/nature_12K/inaturalist_12K/val"

    
    training_data=ImageDataGenerator(rescale=1./255)

    test_data=ImageDataGenerator(rescale=1./255)

    train_dataset = training_data.flow_from_directory(path_train,shuffle=True,seed=19)
    test_dataset = test_data.flow_from_directory(path_test,shuffle=True,seed=19)

    #return train_aug, valid_aug

    
    #return train_data, valid_data
    return train_dataset, test_dataset

In [ ]:
def get_test_augmented_data():
    #path=r"C:\Users\vamsi_oe20s302\Downloads\nature_12K\inaturalist_12K\train"
    path_train = r"/content/drive/MyDrive/nature_12K/inaturalist_12K/train"
    path_test = r"/content/drive/MyDrive/nature_12K/inaturalist_12K/val"    
    training_data_augmentation=ImageDataGenerator(rescale=1./255,
                                        height_shift_range=0.2,
                                        width_shift_range=0.2,
                                        horizontal_flip=True,
                                        zoom_range=0.2,
                                        fill_mode="nearest")

    # test data is not being augmented
    test_data_augmentation=ImageDataGenerator(
        rescale=1./255
        )

    train_aug=training_data_augmentation.flow_from_directory(path_train,shuffle=True,seed=19)
    test_aug=test_data_augmentation.flow_from_directory(path_test,shuffle=True,seed=19)

    return train_aug, test_aug

In [ ]:
def testing(entity_name, project_name):

    best_hyperparameters = {
        "model_name": "Xception",
        "data_augmentation": True,
        "dense_layer": 128,
        "dropout": 0.3,
        "trainable_layers": 20,
        "batch_size": 64,
        "num_epochs": 15
        
    }

    # Initialize a new wandb run
    wandb.init(config=best_hyperparameters, project = project_name, entity=entity_name)
    
    # collecting the configuration information
    config = wandb.config

    # Local variables, values obtained from wandb config
    model_name = config.model_name
    data_augmentation = config.data_augmentation
    dense_layer = config.dense_layer
    dropout = config.dropout
    trainable_layers = config.trainable_layers
    batch_size = config.batch_size
    num_epochs = config.num_epochs
    dense_activation = "relu"
    
    
    # run name
    run_name='test run - partb'

    wandb.run.name = run_name

    # Create the data generators

    if data_augmentation == False:
        train_data, test_data = get_test_data()
    else:
        train_data, test_data = get_test_augmented_data()
    
    # Define the model
    model = build_model(model_name, dense_activation, dense_layer, dropout, trainable_layers)

    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                loss=tf.keras.losses.CategoricalCrossentropy(name='loss'),
                metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')])


    # To save the model with best validation accuracy
    #mc = ModelCheckpoint('/content/drive/MyDrive/nature_12K/best_model.h5', monitor='val_accuracy', mode='max', verbose=0, save_best_only=True)
    wandb_log = True
    model.save("best_pretrained_model.h5")
    history = model.fit(train_data,
                        epochs=num_epochs, 
                        callbacks=[WandbCallback()]
                        )
    
    test_loss, test_accuracy = model.evaluate(x = test_data)
    print(test_loss)
    print(test_accuracy)
    wandb.log({'test_accuracy':test_accuracy, 'test_loss':test_loss})
    
    
    # Meaningful name for the run

    
    wandb.run.finish()
    return history

In [ ]:
testing(entity_name, project_name)

wandb: Currently logged in as: safikhan (use `wandb login --relogin` to force relogin)


Found 9999 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.
83697664/83683744 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 8, 8, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 131072)            0         
                                                                 
 dense (Dense)               (None, 128)               16777344  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 37,640,114
Trainable pa

acc,▁▄▅▅▆▆▆▇▇▇▇████
epoch,▁▁▂▃▃▃▄▅▅▅▆▇▇▇█
loss,█▅▄▄▃▃▂▂▂▂▂▁▁▁▁
test_accuracy,▁
test_loss,▁
acc,0.9548
epoch,14
loss,0.13893
test_accuracy,0.7965
test_loss,1.13256
